# Dining Forecast: FSQ Foot Traffic Data + Apptopia’s Mobile App Data

### Product status
Published.
### Product visibility
Public.
### Description

This analysis examines consumers’ behavior around dining based on foot traffic data from millions of Americans that make up Foursquare’s always-on panel, combined with Apptopia’s data on the usage of mobile apps.


Using data from January 2019 through July 2020 on a national level, you can identify the year over year changes in daily active users of restaurants’ mobile apps, as well as year over year changes in visitation to quick service restaurants and casual dining restaurants, in order to isolate the effects of COVID-19 and to account for seasonality. For foot traffic data, rolling 7-day averages  are leveraged to account for fluctuations by day of week. For mobile app usage data, we looked at weekly sums. 


Index the data, rather tan looking at the overall volume of visits or montly active users, to uncover relative upticks and declines, adjusting for changes in panel size. Index to first week. In order to index, take the nominal volumn of a given week and divide that volume by the volume of the first week of the timeframe. Multiply that product by 100.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
path = '../../data/raw/restaurants/Foursquare_Apptopia_datasets/Foursquare_Apptopia_Dining_Data_2020.xlsx'

In [3]:
xls = pd.ExcelFile(path)

xls.sheet_names

['YoY Apptopia Usage',
 'YoY Foursquare Visits',
 '2019 Foursquare Visits Indexed',
 '2020 Foursquare Visits Indexed',
 '2019 Apptopia Usage Indexed',
 '2020 Apptopia Usage Indexed',
 'YoY Foursquare Visits - Categor',
 'YoY Apptopia Usage - Categories',
 '2019 Foursquare Visits Indexed ',
 '2020 Foursquare Visits Indexed ',
 '2019 Apptopia Usage Indexed - C',
 '2020 Apptopia Usage Indexed - C']

Here there are 6 xls with information by category (casual dining, fast casual dining, coffee, pizza and sandwiches) and 6 files with individual information from different restaurants.

### 1. Split datasets into two groups: 
- Information of restaurants by category.
- Information of restaurant as entity.

In [4]:
df_category = pd.DataFrame()
# look for a key word to find datasets with columns by category. Example, the column casual_dining
aux = 'casual_dining'

for sheet_name in xls.sheet_names:
    temp = pd.read_excel(xls, sheet_name=sheet_name, index_col=0)
    temp['class'] = sheet_name
    if aux in list(temp.columns):
        df_category = pd.concat([df_category, temp], sort=True)

In [5]:
df_category.head()

,casual_dining,class,coffee,fast_casual_food,pizza,sandwiches
week,,,,,,
2,0.000000,YoY Foursquare Visits - Categor,0.000000,0.000000,0.000000,0.000000
3,-11.517221,YoY Foursquare Visits - Categor,-5.539010,-4.084100,-4.373698,2.375718
4,-14.392250,YoY Foursquare Visits - Categor,-8.189281,-4.662596,-3.402129,1.621992
5,-10.447774,YoY Foursquare Visits - Categor,-7.425742,-4.269823,-8.602007,0.900771
6,-12.639638,YoY Foursquare Visits - Categor,-7.537867,-2.541858,-0.163634,4.459046


In [6]:
df_category.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168 entries, 2 to 29
Data columns (total 6 columns):
casual_dining       168 non-null float64
class               168 non-null object
coffee              168 non-null float64
fast_casual_food    168 non-null float64
pizza               168 non-null float64
sandwiches          168 non-null float64
dtypes: float64(5), object(1)
memory usage: 9.2+ KB


In [7]:
df_category.groupby('class').count()

,casual_dining,coffee,fast_casual_food,pizza,sandwiches
class,,,,,
2019 Apptopia Usage Indexed - C,28,28,28,28,28
2019 Foursquare Visits Indexed,28,28,28,28,28
2020 Apptopia Usage Indexed - C,28,28,28,28,28
2020 Foursquare Visits Indexed,28,28,28,28,28
YoY Apptopia Usage - Categories,28,28,28,28,28
YoY Foursquare Visits - Categor,28,28,28,28,28


Saving the dataset in the `interim` folder:

In [8]:
df_category.to_csv('../../data/interim/restaurants/foursquare_apptopia_category.csv')

### 2. From datasets of restaurants as entity:
- Check names of the restaurant match in the Foursquare data and Apptopia data

In [9]:
df1 = pd.read_excel(xls, sheet_name='YoY Apptopia Usage', index_col=0)
df1.columns

Index(['Applebee’s (App)', 'Buffalo Wild Wings (App)',
       'BURGER KING® App (App)', 'Chick-fil-A (App)', 'Chilis (App)',
       'Chipotle (App)', 'Cracker Barrel (App)', 'Dairy Queen® (App)',
       'Domino's Pizza USA (App)', 'Dunkin' (App)',
       'El Pollo Loco - Loco Rewards (App)', 'Firehouse Subs App (App)',
       'Five Guys Burgers & Fries (App)', 'Jack in the Box® (App)',
       'Jersey Mike's (App)', 'Jimmy John’s Sandwiches (App)',
       'Krispy Kreme ® (App)', 'Little Caesars (App)',
       'LongHorn Steakhouse® (App)', 'McDonald's (App)',
       'Olive Garden Italian Kitchen (App)', 'Outback (App)',
       'Panera Bread (App)', 'Papa John's Pizza (App)', 'Peet’s Coffee (App)',
       'Pizza Hut - Delivery & Takeout (App)', 'Popeyes® (App)',
       'Shake Shack (App)', 'SONIC Drive-In (App)', 'Starbucks (App)',
       'SUBWAY® (App)', 'Taco Bell - Food & Rewards (App)',
       'Tim Hortons (App)', 'Wendy’s (App)', 'Whataburger (App)',
       'Wingstop (App)', 'Zaxby’s

In [10]:
df2 = pd.read_excel(xls, sheet_name='YoY Foursquare Visits', index_col=0)
df2.columns

Index(['Applebee's', 'Buffalo Wild Wings', 'Burger King', 'Chick-fil-A',
       'Chili's Grill & Bar', 'Chipotle Mexican Grill', 'Cracker Barrel',
       'Dairy Queen', 'Domino's Pizza', 'Dunkin'', 'El Pollo Loco',
       'Firehouse Subs', 'Five Guys', 'Jack in the Box', 'Jersey Mike's Subs',
       'Jimmy John's', 'Krispy Kreme Doughnuts', 'Little Caesars',
       'LongHorn Steakhouse', 'McDonald's', 'Olive Garden',
       'Outback Steakhouse', 'Panera Bread', 'Papa John's Pizza',
       'Peet's Coffee & Tea', 'Pizza Hut', 'Popeyes Louisiana Kitchen',
       'Shake Shack', 'Sonic Drive-In', 'Starbucks', 'Subway', 'Taco Bell',
       'Tim Hortons', 'Wendy's', 'Whataburger', 'Wingstop', 'Zaxby's'],
      dtype='object')

As we can see, the names are similar but not the same. Apptopia datasets include the expression 'App' in the name of the columns and some spetial characters. To concat those datasets avoiding duplicate columns pointed to the same restaurants, we can create a preprocessing text method to delete spetial characters and patterns, but in this case is not necessary, because the restaurants in the columns have exactly the same order in the Foursquare and Apptopia data. Therefore, we define canonical names for columns before concatenate the datasets:

In [11]:
canonical_columns = df2.columns

And then, we inspect the xls sheet names looking for the datasets related to individual restaurants:

In [12]:
df = pd.DataFrame()

aux = 'casual_dining'

for sheet_name in xls.sheet_names:
    temp = pd.read_excel(xls, sheet_name=sheet_name, index_col=0)
    if aux not in list(temp.columns):
        temp.columns = canonical_columns
        temp['class'] = sheet_name
        df = pd.concat([df, temp], sort=True) 

In [13]:
df.head(2)

,Applebee's,Buffalo Wild Wings,Burger King,Chick-fil-A,Chili's Grill & Bar,Chipotle Mexican Grill,Cracker Barrel,Dairy Queen,Domino's Pizza,Dunkin',...,Sonic Drive-In,Starbucks,Subway,Taco Bell,Tim Hortons,Wendy's,Whataburger,Wingstop,Zaxby's,class
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,YoY Apptopia Usage
3,-8.499222,2.254202,3.855614,29.164418,12.845222,0.660654,9.896487,-5.41138,-2.481952,2.424968,...,0.467718,-0.699329,0.812795,0.908158,0.607167,1.081407,3.363334,-5.779505,34.021747,YoY Apptopia Usage


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214 entries, 2 to 29
Data columns (total 38 columns):
Applebee's                   214 non-null float64
Buffalo Wild Wings           214 non-null float64
Burger King                  214 non-null float64
Chick-fil-A                  214 non-null float64
Chili's Grill & Bar          214 non-null float64
Chipotle Mexican Grill       214 non-null float64
Cracker Barrel               214 non-null float64
Dairy Queen                  214 non-null float64
Domino's Pizza               214 non-null float64
Dunkin'                      214 non-null float64
El Pollo Loco                214 non-null float64
Firehouse Subs               214 non-null float64
Five Guys                    214 non-null float64
Jack in the Box              214 non-null float64
Jersey Mike's Subs           214 non-null float64
Jimmy John's                 214 non-null float64
Krispy Kreme Doughnuts       214 non-null float64
Little Caesars               214 non-null floa

Saving the dataset in the `interim` folder:

In [15]:
df.to_csv('../../data/interim/restaurants/foursquare_apptopia_restaurants.csv')